# Level 4: RAG Agent

This notebook presents an example of executing queries with a RAG agent in Llama Stack. It shows how to initialize an agent with the RAG tool provided by Llama Stack and to invoke it such that retrieval from a vector DB is activated only when necessary. The notebook also covers document ingestion using the RAG tool.

For a simple (non-agentic) RAG tutorial, please refer to [Level1_simple_RAG.ipynb](./Level1_simple_RAG.ipynb).

## Overview

This notebook covers the following steps:

1. Connecting to a llama stack server.
2. Indexing a collection of documents into a vector DB for later retrieval.
3. Initializing the agent capable of retrieving content from vector DB via tool use.
4. Launching the agent and using it to answer user queries about the documents.


## Prerequisites

Before starting, ensure you have the following:
- Followed the instructions in the [Setup Guide](./Level0_getting_started_with_Llama_Stack.ipynb) notebook. 
- Llama Stack server should be using milvus as its vector DB provider.


## 1. Setting Up this Notebook
We will start with a few imports needed for this demo only.

In [1]:
import uuid

from llama_stack_client import Agent, RAGDocument
from llama_stack_client.lib.agents.event_logger import EventLogger

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [2]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server

model_id =os.getenv("MODEL_NAME")

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: llama3.2:3b-instruct-fp16
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 512}
	stream: False


Finally, we will create a unique name for the document collection to be used for RAG ingestion and retrieval.

## 2. Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle the fetching, conversion and chunking of the documents' content automatically.

In [3]:
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=os.getenv("VDB_EMBEDDING"),
    embedding_dimension=int(os.getenv("VDB_EMBEDDING_DIMENSION", 384)),
    provider_id=os.getenv("VDB_PROVIDER"),
)

# ingest the documents into the newly created document collection
urls = [
    ("https://www.openshift.guide/openshift-guide-screen.pdf", "application/pdf"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512)),
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-dbs "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 200 OK"


## 3. Executing queries via the RAG agent
- Initialize an agent with a list of tools including the built-in RAG tool. The RAG tool specification must include a list of document collection IDs to retrieve from.
- For each prompt, initialize a new agent session, execute a turn during which a retrieval call may be requested, and output the reply received from the agent.

In [4]:
queries = [
    "How to install OpenShift?",
]

# initializing the agent
agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant. You must use the knowledge search tool to answer user questions.",
    sampling_params=sampling_params,
    # we make our agent aware of the RAG tool by including builtin::rag/knowledge_search in the list of tools
    tools=[
        dict(
            name="builtin::rag",
            args={
                "vector_db_ids": [vector_db_id],  # list of IDs of document collections to consider during retrieval
            },
        )
    ],
)

for prompt in queries:
    cprint(f"\nUser> {prompt}", "blue")
    
    # create a new turn with a new session ID for each prompt
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=agent.create_session(f"rag-session_{uuid.uuid4()}"),
        stream=stream,
    )
    
    # print the response, including tool calls output
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Arag "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/1c2207b2-9c9e-47fc-8040-998fb992677a/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/1c2207b2-9c9e-47fc-8040-998fb992677a/session/52ea4d25-78c1-4986-9f20-c3f8c5b3c1fb/turn "HTTP/1.1 200 OK"



User> How to install OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: knowledge_search, Arguments: {'query': 'OpenShift installation instructions'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 1\nContent:  "Import from Git" entry. Click on it, and paste the URL of a project, for example,\ngitlab.com/akosma/simple-deno-api.git.\nAs soon as you paste the URL, OpenShift will immediately analyze the structure and programming\nlanguage of the project and automatically recommend options for its build process. In our case, it’s\na small application built with the Go programming language, and as such, it will advise the options\nshown on the screen.\nFigure 5. Deploying a project directly from its Git repository\n25\nThis particular example doesn’t require more configurations than the ones shown on the screen;\nclick the [\u2009Create\u2009] button.\nAfter a few seconds, you will see your application running on the "Topology" screen. OpenShift will\ndownload the source code and trigger your project’s build. Click on the Topology screen icon to see\nthe "Build" section, indicating that a build is running. The compilation and deployment of your\napplication can take some time, depending on the complexity of the source code and the\nprogramming language used.\nOnce the build has finished, on the same pane, you will see a route available under the "Routes"\nsection. Click on it, and you will see your application in action.\n10.2. Container Registry\nOpenShift has built your application source code, and the product of this build process is a\ncontainer. You can see the container that OpenShift made for you on the "Administrator"\nperspective, selecting the "Builds" menu and then the "ImageStreams" menu entry.\nOpenShift includes a container registry; developers can use it as any other registry from outside the\ncluster. Let us use "podman" to access the container registry and run the container locally on your\nworkstation.\nUsers must have the "registry-editor" and the "system:image-builder" roles to access the container\nregistry. Since we’re connected to the Web Console using the "kubeadmin" user, we can provide\nthose roles directly from the user interface without using the command line.\nNavigate to the "User Management" section and select "RoleBindings." Click on the [\u2009Create\nbinding\u2009] button, and fill the form using the following values:\n• Name: developer-sourcecode-registry-editor\n• Namespace: sourcecode\n• Role name: registry-editor\n• Subject: User\n• Subject name: developer\nDo the same for the "system:image-builder" role, using a different "Name" field\nMetadata: {\'document_id\': \'num-0\'}\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text="Result 2\nContent:  over 28 years of experience, currently working as\nSenior Architect for Red Hat. He is a published author, trainer, and speaker. He has written many\nbooks about software development and has shipped cloud, mobile, and desktop apps since 1996.\nAdrian holds a Master in Information Technology from the University of Liverpool.\nAdrian is the co-creator of De Programmatica Ipsum , a monthly publication about the impact of\nsoftware in society.\nWhen not coding or teaching, Adrian likes to spend time with his wife Claudia, his cat Max and his\nOlivetti Lettera 22 typewriter.\n1\nChapter 1. Course Requirements\nThis section enumerates some knowledge and tools that would be useful to follow this course about\nRed Hat OpenShift.\n1.1. Audience\nThis course is geared towards developers interested in running applications in OpenShift. It is also\nuseful for DevOps engineers who want to learn how to use OpenShift to build and deploy cloud\nnative applications.\n1.2. Recommended Tools\nHere’s some basic tooling to try Red Hat OpenShift by yourself:\n• Operating System\n◦ Microsoft Windows, Apple macOS, or Linux.\n• Applications\n◦ Git\n◦ Podman, Podman Desktop, Docker, or Docker Desktop\n• Accounts\n◦ A Red Hat Developer account\n◦ A GitHub account\n1.3. Knowl


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the provided text, here is a summary of the key points related to OpenShift templates and odo tool:

**Templates**

* Templates are used to define reusable and parameterized application configurations for OpenShift.
* Templates can be used to simplify and automate the creation of complex applications, such as microservices.
* Templates can help share best practices and common patterns for deploying applications on OpenShift.
* A template definition is represented by a YAML file (e.g., `00-postgresql-template.yaml`).
* Parameters can be defined in the template, which can be used to customize the application configuration.

**odo Tool**

* The odo tool is a command-line tool provided by Red Hat for creating applications on OpenShift.
* Devfiles are used with the odo tool, which contain information about an application's programming language, dependencies, and other essential details.
* The odo tool supports various

## Key Takeaways
This notebook demonstrated how to implement a RAG agent with Llama Stack. We did this by creating an agent and giving it access to the builtin RAG tool, then invoking the agent on the specified query.



#### Any Feedback?

If you have any feedback on this or any other notebook in this demo series we'd love to hear it! Please go to https://www.feedback.redhat.com/jfe/form/SV_8pQsoy0U9Ccqsvk and help us improve our demos. 